In [16]:
import requests
import json
import csv

In [17]:
# http://www.datamuse.com/api/
def datamuse(query):
    url = "https://api.datamuse.com/words?" + query
    response = requests.get(url)
    return json.loads(response.content)

In [22]:
def mc28():
    with open('sim-eval-master/datasets/mc.csv', newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=';')
        rows = []
        for row in reader: rows.append(row)
        return rows # [['automobile', 'car', '3.92'], ['gem', 'jewel', '3.84'], ... ]    